In [ ]:
import edgar_downloader as ed

In [ ]:
ed.download_files_10k('aapl', r'C:\Edgar_EC\test_output', 'gregsmith@kubickgroup.com', min_date='2010-01-01')

In [ ]:
import edgar_cleaner as ec

In [ ]:
ec.write_clean_html_text_files(r'C:\Edgar_EC\test_output', r'C:\Edgar_EC\test_output_2')

In [ ]:
import ref_data as ref_data

In [ ]:
import edgar_sentiment_wordcount as esw

In [ ]:
esw.write_document_sentiments(r'C:\Edgar_EC\test_output_2', r'C:\Edgar_EC\sentiment_factors.csv')

In [ ]:
ref_data.get_yahoo_data('2010-01-01', '2023-01-01', 'AAPL')

In [ ]:
import edgar_sentiment_analysis_prep as esap

In [ ]:
def full_train_dataset(): # input_min_date = None, input_max_date = None
    '''
    Creates a csv to specified file path
    '''

    tickers_sp100 = ref_data.get_sp100()
    #ed.download_files_10k(‘AAPL’, ‘C:/10k_filings_raw’)
    ed.full_download(["AAPL", "MSFT", "AMZN", "BRK-B", "GOOGL", "UNH", "GOOG", "JNJ", "XOM", "JPM", "NVDA", "PG", "V", "HD", "TSLA"], r'C:\10k_filings_raw', 'gregsmith@kubrickgroup.com', min_date = '2022-01-01', report = '10-K') #min_date = input_min_date, max_date = input_max_date,

    ec.write_clean_html_text_files(r'C:\10k_filings_raw', r'C:\10k_filings_clean')
    


In [ ]:
def full_train_dataset():
   #df_returns = rf.get_yahoo_data(input_min_date, input_max_date, tickers_sp100, 'daily')
    df_returns = ref_data.get_yahoo_data('2022-01-01', '2023-08-01', ["AAPL", "MSFT", "AMZN", "BRK-B", "GOOGL", "UNH", "GOOG", "JNJ", "XOM", "JPM", "NVDA", "PG", "V", "HD", "TSLA"]) # Need to decide dates as I have to pass a date here
    df_returns.to_csv(r'C:\Edgar_EC\test_output\stock_returns_daily.csv', index=False)
    sentiment_dict = ref_data.get_sentiment_word_dict()


In [ ]:

    esw.write_document_sentiments(r'C:\10k_filings_clean', r'C:\Edgar_EC\test_output\sentiment_factors.csv')


In [ ]:
import pandas as pd

def full_train_dataset():
    # Load Data
    stock_returns_daily_df = pd.read_csv(r'C:\Edgar_EC\test_output\stock_returns_daily.csv')                         # Load in stock return data
    sentiment_factors_df = pd.read_csv(r'C:\Edgar_EC\test_output\sentiment_factors.csv')                             # Load in sentiment word count data
    #stock_returns_daily_df = pd.read_csv('C:/EDGAR/example_shares_output2.csv')   
    #sentiment_factors_df = pd.read_csv('C:/EDGAR/example_sentiment_analysis.csv')  
    
    # Processing and Feature Engineering
    full_stock_returns_daily_df = esap.stock_returns_prep(stock_returns_daily_df)
    full_sentiment_factors_df = esap.sentiment_factors_prep(sentiment_factors_df)
    
    # Combine Datasets
    full_combined_df = pd.merge(full_stock_returns_daily_df, full_sentiment_factors_df, on = ['Date', 'ticker'], how = 'left')

    full_combined_df.to_csv(r'C:\Edgar_EC\test_output\full_dataset.csv', index = False)              # Will need to decide a better place for this

full_train_dataset()